In [8]:
file_name = "VSD"
class_name = "complex"

In [ ]:
import os
import SimpleITK as sitk

# Generate MHD sigmoid mask

indir = "./segmentations/" + file_name + "/";
outdir = "./segmentations/" + file_name + "/mhd/";

print(indir)
print(outdir)

if not os.path.exists(outdir) :
    os.makedirs(outdir)
    
for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file)
        
        path = outdir + file.replace(".nii.gz", ".mhd")
        scale_filter = sitk.ShiftScaleImageFilter()
        scale_filter.SetScale(100)
        res = scale_filter.Execute(img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

Mesh generation from MHD voxel files using FAST extraction

In [ ]:
from stl import mesh as stmesh
import math
import numpy as np
import fast

indir = "./segmentations/" + file_name + "/mhd/";
outdir = "./meshes/stl";

# Spine
#files = [\
#    "vertebrae_L1",\
#    "vertebrae_L2",\
#    "vertebrae_L3",\
#    "vertebrae_L4",\
#    "vertebrae_L5",\
#    "vertebrae_T1",\
#    "vertebrae_T10",\
#    "vertebrae_T11",\
#    "vertebrae_T12",\
#    "vertebrae_T2",\
#    "vertebrae_T3",\
#    "vertebrae_T4",\
#    "vertebrae_T5",\
#    "vertebrae_T6",\
#    "vertebrae_T7",\
#    "vertebrae_T8",\
#    "vertebrae_T9",\
#    "sacrum"];

# Heart
#files = [\
#    "heart_atrium_left",\
#    "heart_atrium_right",\
#    "heart_myocardium",\
#    "heart_ventricle_left",\
#    "heart_ventricle_right"];

# Aorta
files = ["aorta"]

verts = []
faces = []

offset = 0

for file in files:
    importer = fast.ImageFileImporter.create(indir + file + ".mhd")
    smoothing = fast.GaussianSmoothing.create(stdDev=2.0).connect(importer)
    extraction = fast.SurfaceExtraction.create(threshold=50).connect(smoothing)
    mesh = extraction.runAndGetOutputData()
    access = mesh.getMeshAccess(fast.ACCESS_READ)

    for vertex in access.getVertices():
        verts.append(vertex.getPosition().reshape(1, 3))

    for face in access.getTriangles():
        faces.append([offset + face.getEndpoint1(), offset + face.getEndpoint2(), offset + face.getEndpoint3()])
        
    offset = len(verts)
    
    print(file)
    
verts = np.array(verts)
faces = np.array(faces)
    
model = stmesh.Mesh(np.zeros(faces.shape[0], dtype=stmesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        model.vectors[i][j] = verts[f[j],:]
    
model.save(outdir + "output.stl")

In [ ]:
from vmtk import pypes
from vmtk import vmtkscripts
import pipes

args = "vmtkimagereader -ifile ./segmentations/" + file_name + "/mhd/" + class_name + ".mhd --pipe vmtkimagewriter -ofile ./segmentations/" + file_name + "/vti/" + class_name + ".vti"
                                                                
res = pypes.PypeRun(args)

In [ ]:
# Segmenting the aorta via levelsets

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./segmentations/" + file_name + "/mhd/" + class_name + ".mhd --pipe vmtklevelsetsegmentation -ofile ./meshes/vtp/" + class_name + ".vtk"

pypes.PypeRun(args)

In [ ]:
# Clipping mesh inlets and outlets
  
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfaceclipper -ifile ./meshes/vtp/" + class_name + ".vtk -ofile ./meshes/vtp/" + class_name + "_clip.vtk"

res = pypes.PypeRun(args)

In [ ]:
# Surface smoothing

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacesmoothing -ifile ./meshes/vtp/" + class_name + "_clip.vtp -passband 0.1 -iterations 30 -ofile ./meshes/vtp/" + class_name + "_smooth.vtk"

res = pypes.PypeRun(args)

In [ ]:
# Surface subdivision

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacesubdivision -ifile ./meshes/vtp/" + class_name + "_smooth.vtk -ofile ./meshes/vtp/" + class_name + "_subdiv.vtk -method butterfly"

res = pypes.PypeRun(args)

In [ ]:
# Comparison

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/" + class_name + "_clip.vtk --pipe vmtksurfacesmoothing -iterations 30 -passband 0.1 --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtksurfaceviewer -i @vmtksurfacereader.o -color 1 0 0 -display 1"

res = pypes.PypeRun(args)

In [ ]:
# Centerline computation

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkcenterlines -ifile ./meshes/vtp/" + class_name + "_smooth.vtk -ofile ./meshes/vtp/" + class_name + "_centerline.vtk"

res = pypes.PypeRun(args)

In [ ]:
# Centerline computation visualization

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/" + class_name + "_smooth.vtk --pipe vmtkcenterlines --pipe vmtkrenderer --pipe vmtksurfaceviewer -opacity 0.25 --pipe vmtksurfaceviewer -i @vmtkcenterlines.voronoidiagram -array MaximumInscribedSphereRadius --pipe vmtksurfaceviewer -i @vmtkcenterlines.o"

res = pypes.PypeRun(args)

Centerline extraction using skeletonization with voxel structures

In [ ]:
# Alternative voxel-based centerline extraction

import SimpleITK as sitk
import os
import numpy as np
import time
from scipy.ndimage import binary_erosion

file = "./segmentations/" + file_name + "/mhd/" + class_name ".mhd"

img = sitk.ReadImage(file)

array = sitk.GetArrayFromImage(img)

eroded = array

start = time.perf_counter()

print(np.max(array))

for i in range(0, 10):
    eroded = binary_erosion(eroded)
    array += eroded
    
end = time.perf_counter()

maxval = np.max(array)

print(end - start)
print(maxval)

centerline = []

size = array.shape

for x in range(0, size[0]):
    for y in range(0, size[1]):
        for z in range(0, size[2]):
            if (array[x][y][z] == maxval):
                centerline.append((x, y, z))

Mesh extraction using GMSH from prepared VTK surface mesh

In [ ]:
import os
import vtk

outdir = "./meshes/";

filepath = "./meshes/vtp/" + class_name + "_smooth.vtk"

verts = []
faces = []

offset = 0

basename = os.path.basename(filepath)
print("Copying file:", basename)
basename = os.path.splitext(basename)[0]
outfile = os.path.join(outdir, basename+".stl")
reader = vtk.vtkGenericDataObjectReader()
reader.SetFileName(filepath)
reader.Update()
writer = vtk.vtkSTLWriter()
writer.SetInputConnection(reader.GetOutputPort())
writer.SetFileName(outfile)
writer.Write()

In [ ]:
import gmsh
import sys
import os
import math

gmsh.initialize(sys.argv)

gmsh.merge(os.path.join('./meshes/vtp/aorta.stl'))
gmsh.model.mesh.classifySurfaces(math.pi, True, True)
gmsh.model.mesh.createGeometry()

gmsh.option.setNumber('Geometry.ExtrudeReturnLateralEntities', 0)

gmsh.model.geo.extrudeBoundaryLayer(gmsh.model.getEntities(2), [4], [1.0], True)

e = gmsh.model.geo.extrudeBoundaryLayer(gmsh.model.getEntities(2), [4], [-1.0],
                                        True, True)

top_ent = [s for s in e if s[0] == 2]
top_surf = [s[1] for s in top_ent]

gmsh.model.geo.synchronize()
bnd_ent = gmsh.model.getBoundary(top_ent)
bnd_curv = [c[1] for c in bnd_ent]

loops = gmsh.model.geo.addCurveLoops(bnd_curv)
for l in loops:
    top_surf.append(gmsh.model.geo.addPlaneSurface([l]))

gmsh.model.geo.addVolume([gmsh.model.geo.addSurfaceLoop(top_surf)])
gmsh.model.geo.synchronize()

gmsh.option.setNumber('Mesh.Algorithm', 1)
gmsh.option.setNumber('Mesh.MeshSizeFactor', 0.1)

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()

In [ ]:
# Load surface mesh
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/" + class_name + "_smooth.vtp --pipe vmtkcenterlines -endpoints 1 --pipe vmtkdistancetocenterlines -useradius 1 -ofile ./meshes/vtp/" + class_name + "_centerdist.vtp"

res = pypes.PypeRun(args)

In [ ]:
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfaceviewer -ifile ./meshes/vtp/" + class_name + "_centerdist.vtp -array DistanceToCenterlines"

res = pypes.PypeRun(args)

In [55]:
from vmtk import vmtkscripts
import numpy as np

def surface_to_numpy(class_name):
    reader = vmtkscripts.vmtkSurfaceReader()
    np_adpt = vmtkscripts.vmtkSurfaceToNumpy()

    reader.InputFileName = "./meshes/vtp/" + class_name + "_centerdist.vtp"
    reader.Execute()
    np_adpt.Surface = reader.Surface
    np_adpt.Execute()

    mesh_dict = np_adpt.ArrayDict
    
    return mesh_dict
    
def numpy_to_surface(mesh_dict):
    surf_adpt = vmtkscripts.vmtkNumpyToSurface()
    surf_adpt.ArrayDict = mesh_dict
    surf_adpt.Execute()

    writer = vmtkscripts.vmtkSurfaceWriter()
    writer.Surface = surf_adpt.Surface
    writer.OutputFileName = "./meshes/vtp/" + class_name + "_adaptive.vtp"
    writer.Execute()
    
def smooth_array(array, neighbors):
    new_array = np.empty(array.shape)
    print(neighbors[0:10])
    for i in range(0, len(neighbors)):
        total = 0
        for index in neighbors[i]:
            total += array[index]
        if len(neighbors[i]) > 0:
            total /= len(neighbors[i])
        else:
            total = array[i]
        new_array[i] = total
    print(array[0:10])
    print(new_array[0:10])
    return new_array;
    
    
def boundary_layer(class_name):
    mesh_dict = surface_to_numpy(class_name)

    pts = mesh_dict['Points']
    dist = mesh_dict['PointData']['DistanceToCenterlines']
    norm = mesh_dict['PointData']['Normals']
    scal = mesh_dict['PointData']['Scalars_']
    tris = mesh_dict['CellData']['CellPointIds']

    point_count = pts.shape[0]
    new_pts = np.empty(pts.shape)

    print(pts.shape)

    # Iterate over all points in the mesh and extrude them along normals while scaling extrustion
    for i in range(0, point_count):
        vert = pts[i]
        new_vert = vert - norm[i] * dist[i] * 0.2
        new_pts[i] = new_vert 

    mesh_dict['Points'] = np.append(pts, new_pts, axis=0)

    print(mesh_dict['Points'].shape)

    face_count = tris.shape[0]
    new_faces = np.empty(tris.shape)

    print(tris.shape)

    # Iterate over all faces of the mesh and connect them
    for i in range(0, face_count): 
        face = tris[i]
        new_face = np.empty(face.shape)
        new_face[0] = face[0] + point_count
        new_face[1] = face[1] + point_count
        new_face[2] = face[2] + point_count
        new_faces[i] = new_face 

    mesh_dict['CellData']['CellPointIds'] = np.append(tris, new_faces, axis=0)

    print(mesh_dict['CellData']['CellPointIds'].shape)

    mesh_dict['PointData']['DistanceToCenterlines'] = np.append(dist, dist, axis=0)
    mesh_dict['PointData']['Normals'] = np.append(norm, norm, axis=0)
    mesh_dict['PointData']['Scalars_'] = np.append(scal, scal, axis=0)

    numpy_to_surface(mesh_dict)

def boundary_layer_vmr(class_name):
    mesh_dict = surface_to_numpy(class_name)

    pts = mesh_dict['Points']
    dist = mesh_dict['PointData']['DistanceToCenterlines']
    tris = mesh_dict['CellData']['CellPointIds']
    face_norm = mesh_dict['CellData']['Normals']

    point_norm = np.empty(pts.shape, dtype=np.float32)
    point_occurences = np.zeros(pts.shape)

    point_count = pts.shape[0]
    face_count = tris.shape[0]

    neighbors = [set() for i in range(0, point_count)]

    for i in range(0, face_count): 
        point_norm[tris[i][0]] += face_norm[i]
        point_occurences[tris[i][0]] += 1

        point_norm[tris[i][1]] += face_norm[i]
        point_occurences[tris[i][1]] += 1

        point_norm[tris[i][2]] += face_norm[i]
        point_occurences[tris[i][2]] += 1

        for a in range(0, 3):
            for b in range(0, 3):
                neighbors[tris[i][a]].add(tris[i][b])


    for i in range(0, point_count):
        point_norm[i] /= point_occurences[i]

    norm = point_norm

    point_count = pts.shape[0]
    new_pts = np.empty(pts.shape, dtype=np.float32)
    
    # Distance field smoothing
    for i in range(0, 15):
        dist = smooth_array(dist, neighbors)
    
    # Iterate over all points in the mesh and extrude them along normals while scaling extrustion
    for i in range(0, point_count):
        vert = pts[i]
        new_vert = vert - norm[i] * dist[i] * 0.25
        new_pts[i] = new_vert 

    face_count = tris.shape[0]
    new_faces = np.empty(tris.shape, dtype=np.int64)

    print(tris.shape)

    # Iterate over all faces of the mesh and connect them
    for i in range(0, face_count): 
        face = tris[i]
        new_face = [0, 0, 0]
        new_face[0] = int(face[0] + point_count)
        new_face[1] = int(face[1] + point_count)
        new_face[2] = int(face[2] + point_count)
        new_faces[i] = new_face

    new_dict = {}

    new_dict['Points'] = np.append(pts, new_pts, axis=0)
    new_dict['PointData'] = {}
    new_dict['PointData']['Normals'] = np.append(norm, norm, axis=0)
    new_dict['CellData'] = {}

    new_dict['CellData']['CellPointIds'] = np.append(tris, new_faces, axis=0)

    numpy_to_surface(new_dict)

In [56]:
# Extract mesh and generate boundary layer

from vmtk import vmtkscripts
import numpy as np

boundary_layer_vmr("complex")

Reading VTK XML surface file.
wrapping vtkPolyData object
converting cell data: 
GlobalElementID
GlobalBoundaryCells
CapID
BadTriangle
FreeEdge
BooleanRegion
ModelFaceID
Normals
ActiveCells
converting points
converting point data: 
GlobalNodeID
GlobalBoundaryPoints
DistanceToCenterlines
converting cell connectivity list
[{0, 1, 2, 3, 4, 5, 6}, {0, 1, 2, 3, 815, 816, 819}, {0, 1, 2, 6, 67561, 67562, 816, 851}, {0, 1, 3, 4, 1320, 819, 1277}, {0, 3, 4, 5, 1320, 67226}, {0, 67138, 4, 5, 6, 845, 850, 67226}, {0, 2, 5, 6, 850, 851}, {7, 8, 9, 10, 11, 12, 13}, {772, 7, 8, 9, 12, 1305, 1307}, {7, 8, 9, 10, 1307, 67219, 67067, 891}]
[0.20730448 0.21769227 0.21291744 0.20808718 0.20505161 0.19966771
 0.2048323  0.37614526 0.37585259 0.37083279]
[0.20793614 0.22263103 0.22566921 0.20825503 0.20236815 0.20115308
 0.20603325 0.37682694 0.37419535 0.35817075]
[{0, 1, 2, 3, 4, 5, 6}, {0, 1, 2, 3, 815, 816, 819}, {0, 1, 2, 6, 67561, 67562, 816, 851}, {0, 1, 3, 4, 1320, 819, 1277}, {0, 3, 4, 5, 1320, 6

(136022, 3)
converting points
converting numpy array to surface
Writing VTK XML surface file.


In [57]:
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfaceviewer -opacity 0.3 -ifile ./meshes/vtp/" + class_name + "_adaptive.vtp"

res = pypes.PypeRun(args)


Automatic piping vmtksurfaceviewer
Parsing options vmtksurfaceviewer
    SurfaceInputFileName = ./meshes/vtp/complex_adaptive.vtp
    Opacity = 0.3
Explicit piping vmtksurfaceviewer
Input vmtksurfaceviewer members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/complex_adaptive.vtp
    vmtkRenderer = None
    Display = 1
    Representation = surface
    Opacity = 0.3
    ArrayName = 
    ScalarRange = [0.0, 0.0]
    ColorMap = cooltowarm
    NumberOfColors = 256
    Legend = 0
    FlatInterpolation = 0
    DisplayCellData = 0
    DisplayTag = False
    RegionTagArrayName = RegionTagArray
    Color = [-1.0, -1.0, -1.0]
    LineWidth = 1
    LegendTitle = 
    SurfaceOutputFileName = 
Reading VTK XML surface file.
Executing vmtksurfaceviewer ...
Done executing vmtksurfaceviewer.
Output vmtksurfaceviewer members:
    Id = 0
    Surface = vtkPolyData
    Actor = vtkActor
